In [1]:
import nltk, re, pprint
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.collocations import BigramCollocationFinder
from nltk.corpus import stopwords
from collections import Counter
import codecs
import sys
import os
import string
import random
import itertools as it
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
import matplotlib.colors as colors
plot.rcParams['figure.figsize'] = (21, 14)
from scipy.stats import rankdata
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import words
from nltk.corpus import wordnet as wn
from nltk.corpus import reuters

# DATA620: Week 10/11 Assignment

> #### Bryant Chang, Thomas Detzel, Sandipayan Nandi, and Erik Nylander


## 1. Introduction and Results  

For this project, we use the Reuters Corpus ([Reuters 21578](http://disi.unitn.it/moschitti/corpora.htm)) data set that comes packaged with [NLTK](http://www.nltk.org/book/ch02.html). It contains 10,788 business and financial news stories totaling 1.3 million words. The documents have been classified using 90 business topics. For instance, earning reports are flagged 'earn'; commodity price reports are flagged by commodity label ('crude' or 'palm-oil'); economic indicators are labeled by type ('cpi', 'gnp'), etc. (For definitions, see Appendix A.) 

Categories in the Reuters Corpus overlap each other, because a news story often covers multiple topics. Our goal was to build a model with the training dataset to predict the class of new documents in the test dataset. For news articles with multiple labels, we used label listed first to train and test. To experiment, we employed both Naive Bayes and SVM classifiers to train the data. Results showed that SVM had a higher overall classification accuracy (appox. 90 percent) compared to Naive Bayes (84 percent).


##  2.  Naive Bayes with Text Processing
As a first step, we tokenize the text to remove non-words and stop words. Next, we stem the words to remove endings and reduce the word count.

### 2.1 Import and prepare Reuters corpus

In [2]:
## get the reuters docs
documents = [(reuters.raw(fileid), category)
    for category in reuters.categories()
    for fileid in reuters.fileids(category)]

The function below takes a given text and proceds to tokenize the text, remove any non-words from the text, remove any words that are less than 3 characters long, and then procceds to stem the word down to its "root".

In [4]:
## Define a tokenizer that also stems the documents to improve classifier performance.
StopWords = stopwords.words()
 
def tokenize(text):
    min_length = 3
    words = map(lambda word: word.lower(), word_tokenize(text));
    words = [word for word in words
                  if word not in StopWords]
    tokens =(list(map(lambda token: PorterStemmer().stem(token),
                  words)));
    p = re.compile('[a-zA-Z]+');
    filtered_tokens = list(filter(lambda token:
         p.match(token) and len(token)>=min_length,
         tokens));
    return filtered_tokens

In [5]:
## Creating a new document list after passing each document through the tokenizing function.
documents2 = []
for doc in documents:
    documents2.append((tokenize(doc[0]), doc[1]))

In [16]:
# remove stop words from word list and steming words in words list
t_words = [word.lower() for word in reuters.words() if word.isalpha()]
t_words = [w for w in t_words if w not in StopWords]
# t_words = [PorterStemmer().stem(w) for w in t_words]

#### 2.12 View of stemmed word list

In [7]:
list(t_words)[:9]

[u'asian',
 u'export',
 u'fear',
 u'damag',
 u'japan',
 u'rift',
 u'mount',
 u'trade',
 u'friction']

### 2.2 Create a feature extractor

The following steps select the 2,000 most-common words and build a feature extractor that will be used by the classifier. The feature is simple: If one of the most common words is found in a document the feature extractor reports true.

In [17]:
## create list of the 2,000 most common words
all_words = nltk.FreqDist(t_words)
word_features = all_words.most_common(2000)

## get just the words, no counts
word_features = [w for (w,v) in word_features]

In [9]:
all_words.items()[0:9]

[(u'xylen', 2),
 (u'hallwood', 1),
 (u'petrolan', 4),
 (u'surpasss', 1),
 (u'middleman', 2),
 (u'nasouli', 1),
 (u'fawc', 2),
 (u'yellow', 17),
 (u'degussa', 10)]

In [10]:
word_features[:9]

[u'said', u'mln', u'vs', u'dlr', u'pct', u'lt', u'ct', u'year', u'net']

In [11]:
# update feature extractor with new features
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features[word] = (word in document_words)
    return features

### 2.3 Create training and test sets and build classifier

In [18]:
## creating the feature set and trainign and test sets.
featuresets = [(document_features(d), c) for (d, c) in documents2]
train_set, test_set = featuresets[1000:], featuresets[:1000]

In [19]:
## Building the classifier
classifier = nltk.NaiveBayesClassifier.train(train_set)

### 2.4 Testing the Classifier 
Now that we have built our classifier, we test performance on 1,000 documents that were not part of the training set. We find that we get an accuracy of 80%.

In [20]:
print("Accuracy is {}%.".format(nltk.classify.accuracy(classifier, test_set)*100))

Accuracy is 80.3%.


In [25]:
classifier.show_most_informative_features(100)

Most Informative Features
                    palm = True           palm-o : earn   =   2611.1 : 1.0
                 sorghum = True           sorghu : earn   =   2454.5 : 1.0
               economist = True              dfl : earn   =   2312.9 : 1.0
               indonesia = True           copra- : earn   =   2312.9 : 1.0
                    crop = True           copra- : earn   =   1652.1 : 1.0
                  tariff = True              dfl : earn   =   1652.1 : 1.0
              indonesian = True           copra- : earn   =   1652.1 : 1.0
                   cargo = True           propan : earn   =   1321.7 : 1.0
                   saudi = True           propan : earn   =   1321.7 : 1.0
                  arabia = True           propan : earn   =   1321.7 : 1.0
                  bureau = True              rye : earn   =   1321.7 : 1.0
                   grade = True              rye : earn   =   1321.7 : 1.0
                    corn = True             corn : earn   =   1142.9 : 1.0

### 2.5 Discussion

Review of the most informative features shows that, as expected, appearance of a specific commodity name in a document is a strong predictor of commodity category. The two most informative word features are 'palm' and 'sorghum.' But the classifier also picked up on geographic associations. Indonesia, the largest world producer of copra (dried coconut used to extract coconut oil), is the most informative feature for the 'copra' category. Similarly, 'saudi' and 'arabia' are most informative features for the 'propane' category. 

The list of most informative features also shows how error can creep in. Geographic words can associate with multiple categories as can financial terms such as 'quota', which can be applied in multiple news scenarios. 

In [ ]:
***********************

## 3. Support Vector Machine for Text Classification

In this section, we adapt concepts outlined in the [QuantStart blog](https://www.quantstart.com/articles/Supervised-Learning-for-Document-Classification-with-Scikit-Learn) to the Reuters  corpus. This approach uses TF-IDF to streamline and normalize the word feature set and the support vector machine classifier in Scikit-Learn machine learning package for Python. TF-IDF, for term-frequency, inverse document frequency, scales down the importance of non-informative words.

### 3.1 Prepare the corpus and topic list

In [26]:
## get the raw corpus
corpus = []
for id in range(len(nltk.corpus.reuters.fileids())):
    file = nltk.corpus.reuters.raw(fileids=list(nltk.corpus.reuters.fileids())[id])
    corpus.append(file)

In [27]:
## get the topics list for each document in the corpus
topics = []
for id in range(len(nltk.corpus.reuters.fileids())):
    cat = nltk.corpus.reuters.categories(fileids=list(nltk.corpus.reuters.fileids())[id])
    topics.append(cat)

In [141]:
## check results
## len(topics), topics[0:5]

(10788,
 [[u'trade'],
  [u'grain'],
  [u'crude', u'nat-gas'],
  [u'corn', u'grain', u'rice', u'rubber', u'sugar', u'tin', u'trade'],
  [u'palm-oil', u'veg-oil']])

### 3.2 Function to extract first topic from each document

In [28]:
## revised function to take the raw reuters topics and filter out only the first one for each document

def filter_topics(topics):
    
    """
    Selects the first category when
    multiple topics are listed for a
    specific document in the corpus.
    Returns the list of categories.
    
    """
    cats = []
    for i in range(len(topics)):
#        if topics[i][0] == [] or topics[i][0] == "":
#            continue
        if topics[i][0] in nltk.corpus.reuters.categories():
            cat = topics[i][0]
            cats.append(cat)
#                break
    return cats

In [29]:
## run it on the topics and assign to results the variable y for the SVM
y = filter_topics(topics)

### 3.3 Extract features and create test, training sets
We hold back 20 percent of the documents for testing.

In [30]:
## Create the TF-IDF vectoriser and transform the corpus
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=1)
X = vectorizer.fit_transform(corpus)

In [31]:
## get test and train sets
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.2, random_state=42
)

### 3.4 SVM using C-support vector classifcation

In [32]:
## QuantStart function to train the model using SVC classifier
from sklearn.svm import SVC

def train_svm(X, y):
    """
    Create and train the Support Vector Machine.
    """
    svm = SVC(C=1000000.0, gamma='auto', kernel='rbf')
    svm.fit(X, y)
    return svm

## run it
svm = train_svm(X_train, y_train)

In [34]:
from sklearn.metrics import confusion_matrix

## Make an array of predictions on the test set
pred = svm.predict(X_test)

## Output the hit-rate and the confusion matrix for each model
score = svm.score(X_test, y_test)
print ''
print "Accuracy is {}%.".format(score*100)
print ''
## print(confusion_matrix(pred, y_test))


Accuracy is 89.3419833179%.

[[454   0   0 ...,   0   0   1]
 [  0   6   0 ...,   0   0   0]
 [  0   0   5 ...,   0   0   0]
 ..., 
 [  0   0   0 ...,   6   0   0]
 [  0   0   0 ...,   0   1   0]
 [  0   0   0 ...,   0   0   1]]


### 3.5 Classifier using stochastic gradient descent (SGD) learning

In [42]:
import os
import time
import string
import pickle

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report as clsr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split as tts

In [65]:
## Use SDG classifier

def train_sgd(X, y):
    """
    Create and train the Support Vector Machine.
    """
    classifier = SGDClassifier()
    classifier.fit(X, y)
    return classifier

## run it
sgd = train_sgd(X_train, y_train)

In [66]:
from sklearn.metrics import confusion_matrix

## Make an array of predictions on the test set
pred = sgd.predict(X_test)

## Output the hit-rate and the confusion matrix for each model
score = sgd.score(X_test, y_test)
print ''
print "Accuracy is {}%.".format(score*100)
print ''
## print(confusion_matrix(pred, y_test))


Accuracy is 90.4077849861%.



### 3.6 Result Discussion

In this section we elected to try a number of different techniques for training our Reuter’s Classifier from the SVM to bigrams and other classifiers. We have published the two methods that we found that improve the performance of the classifier. The Support Vector Machine uses stochastic gradient descent to find the weights of the various words for predicting each of the categories. This method outperforms the Naive Bayes method with the SVM giving an accuracy of 90.4%. We also see that many of our misclassified documents fall into the trap of being able to be logically classified into many different categories. Our other attempts at modifying the model ultimately led to worse results than our basic word frequency classifier and we find it interesting how good a job of classification a model can achieve with such a basic measure.

***

## Appendix A – Reuters category definitions


Some notes on the Reuters Categories
David D. Lewis
3-Dec-96

  The letter W. Bruce Croft received from Phil Hayes (March 9, 1990)
gave a list of 135 TOPICS categories, which were used in HAYES89.  I
reproduce this list below.

  The email message David D. Lewis received (Nov. 26, 1990) from Phil
Hayes gave lists (which I reproduce below) for categories in these
fields:

                            number stated      number actually 
                            at top of list        on list
  ORGS                           56                  56
  EXCHANGES                      39                  39 
  PLACES (i.e. COUNTRIES)       176                 175
  PEOPLE                        269                 267

    Total                       540                 537
    Total stated in IEEE paper: 539 

  The total 674 categories mentioned in HAYES90b is the sum of 135 and
539.  So the published numbers are a little off. For the purposes of
Reuters-21578 we have taken as ground truth the actual lists of
category names, and the number of items on those lists.
  
*********************************************************************

****Subject Codes (135) 

Money/Foreign Exchange (MONEY-FX)
Shipping (SHIP)
Interest Rates (INTEREST)

**Economic Indicator Codes (16)

Balance of Payments (BOP)
Trade (TRADE)
Consumer Price Index (CPI)
Wholesale Price Index (WPI)
Unemployment (JOBS)
Industrial Production Index (IPI)
Capacity Utilisation (CPU)
Gross National/Domestic Product (GNP)
Money Supply (MONEY-SUPPLY)
Reserves (RESERVES)
Leading Economic Indicators (LEI)
Housing Starts (HOUSING)
Personal Income (INCOME)
Inventories (INVENTORIES)
Instalment Debt/Consumer Credit (INSTAL-DEBT)
Retail Sales (RETAIL)

**Currency Codes (27)

U.S. Dollar (DLR)
Australian Dollar (AUSTDLR)
Hong Kong Dollar (HK)
Singapore Dollar (SINGDLR)
New Zealand Dollar (NZDLR)
Canadian Dollar (CAN)
Sterling (STG)
D-Mark (DMK)
Japanese Yen (YEN)
Swiss Franc (SFR)
French Franc (FFR)
Belgian Franc (BFR)
Netherlands Guilder/Florin (DFL)
Italian Lira (LIT)
Danish Krone/Crown (DKR)
Norwegian Krone/Crown (NKR)
Swedish Krona/Crown (SKR)
Mexican Peso (MEXPESO)
Brazilian Cruzado (CRUZADO)
Argentine Austral (AUSTRAL)
Saudi Arabian Riyal (SAUDRIYAL)
South African Rand (RAND)
Indonesian Rupiah (RUPIAH)
Malaysian Ringitt (RINGGIT)
Portuguese Escudo (ESCUDO)
Spanish Peseta (PESETA)
Greek Drachma (DRACHMA)

**Corporate Codes (2)

Mergers/Acquisitions (ACQ)
Earnings and Earnings Forecasts (EARN)

**Commodity Codes (78)

ALUM
BARLEY
CARCASS
CASTOR-MEAL
CASTOR-OIL
CASTORSEED
CITRUSPULP
COCOA
COCONUT-OIL
COCONUT
COFFEE
COPPER
COPRA-CAKE
CORN-OIL
CORN
CORNGLUTENFEED
COTTON 
COTTON-MEAL
COTTON-OIL
COTTONSEED
F-CATTLE
FISHMEAL
FLAXSEED
GOLD
GRAIN
GROUNDNUT
GROUNDNUT-MEAL
GROUNDNUT-OIL
IRON-STEEL
LEAD
LIN-MEAL
LIN-OIL
LINSEED
LIVESTOCK
L-CATTLE
HOG
LUMBER
LUPIN
MEAL-FEED
NICKEL
OAT
OILSEED
ORANGE
PALLADIUM
PALM-MEAL
PALM-OIL
PALMKERNEL
PLATINUM
PLYWOOD
PORK-BELLY
POTATO
RAPE-MEAL
RAPE-OIL
RAPESEED
RED-BEAN
RICE
RUBBER
RYE
SILK
SILVER
SORGHUM
SOY-MEAL
SOY-OIL
SOYBEAN
STRATEGIC-METAL
SUGAR
SUN-MEAL
SUN-OIL
SUNSEED
TAPIOCA
TEA
TIN
TUNG-OIL
TUNG
VEG-OIL
WHEAT
WOOL
ZINC

**Energy Codes (9)

Crude Oil (CRUDE)
Heating Oil/Gas Oil (HEAT)
Fuel Oil (FUEL)
Gasoline (GAS)
Natural Gas (NAT-GAS)
Petro-Chemicals (PET-CHEM)
Propane (PROPANE)
Jet and Kerosene (JET)
Naphtha (NAPHTHA)

________________________________________________________________

@heading[Organization Codes (56)]
@begin[format]
African Development Bank (ADB-AFRICA)@*
Agency for International Development  (AID)@*
Asian Development Bank  (ADB-ASIA)@*
Association of International Bond Dealers (AIBD)@*
Association of Natural Rubber Producing Countries (ANRPC)@*
Association of South East Asian Nations (ASEAN)@*
Association of Tin Producing Countries (ATPC)@*
Bank for International Settlements (BIS)@*
Council for Mutual Economic Assistance (COMECON)@*
Economic Commission for Africa (ECA)@*
Economic Commission for Asia and the Far East (ECAFE)@*
Economic Commission for Europe (ECE)@*
Economic Commission for Latin America and The Caribbean (ECLA)@*
Economic Commission for West Asia (ECWA)@*
Economic and Social Commission for Asia and the Pacific (ESCAP)@*
European Atomic Energy Community (EURATOM)@*
European Coal and Steel Community (ECSC)@*
European Community (EC)@*
European Free Trade Association (EFTA)@*
European Investment Bank (EIB)@*
European Monetary Cooperation Fund (EMCF)@*
Food and Agriculture Organisation (FAO)@*
General Agreement on Tariffs and Trade (GATT)@*
Grupo de Paises Latinoamericanos y del Caribe Exportadores de Azucar
[Group of Latin American and Caribbean Sugar Exporting Countries]
(GEPLACEA)@*
Gulf Cooperation Council (GCC)@*
Inter-Government Council of Copper Exporting Countries (CIPEC)@*
Inter-Governmental Maritime Consultative Organisation (IMCO)@*
International Air Transport Association (IATA)@*
International Atomic Energy Authority (IAEA)@*
International Bank for Reconstruction and Development (WORLDBANK)@*
International Cocoa Organisation (ICCO)@*
International Coffee Organisation (ICO-COFFEE)@*
International Development Association (IDA)@*
International Energy Agency (IEA)@*
International Iron and Steel Institute (IISI)@*
International Labour Organisation (ILO)@*
International Lead and Zinc Study Group (ILZSG)@*
International Monetary Fund (IMF)@*
International Natural Rubber Organisation (INRO)@*
International Rubber Study Group (IRSG)@*
International Sugar Agreement (ISA)@*
International Tin Council (ITC)@*
International Whaling Commission (IWC-WHALE)@*
International Wheat Council (IWC-WHEAT)@*
International Wool Secretariat (IWS)@*
International Wool Textile Organisation (IWTO)@*
International Wrought Copper Council (IWCC)@*
Islamic Conference Organisation (ICO-ISLAM)@*
Latin American Free Trade Association (LAFTA)@*
Multi-Fibres Arrangement (MFA)@*
Organisation for Economic Cooperation and Development (OECD)@*
Organisation of Arab Petroleum Exporting Countries (OAPEC)@*
Organisation of Petroleum Exporting Countries (OPEC)@*
United Nations (UN)@*
United Nations Conference on Trade and Development (UNCTAD)@*
World Health Organisation (WHO)@*
@end[format]
@blankspace[3 lines]
________________________________________________________________
@heading[Exchange Codes (39)]
@begin[format]
American Stock Exchange (AMEX)@*
Amsterdam Stock Exchange/Bourse (ASE)@*
Australian Stock Exchange (ASX)@*
Baltic International Freight Futures Exchange (BIFFEX)@*
Brussels Stock Exchange/Bourse (BSE)@*
Chicago Board of Trade (CBT)@*
Chicago Board of Trade Options Exchange (CBOE)@*
Chicago Mercantile Exchange (CME)@*
Copenhagen Stock Exchange/Bourse (CSE)@*
Frankfurt Stock Exchange/Bourse (FSE)@*
Hong Kong Stock Exchange (HKSE)@*
International Petroleum Exchange (IPE)@*
Johannesburg Stock Exchange (JSE)@*
Kuala Lumpur Commodity Exchange (KLCE)@*
Kuala Lumpur Stock Exchange (KLSE)@*
London Futures and Options Exchange (FOX)@*
London International Financial Futures Exchange (LIFFE)@*
London Metal Exchange (LME)@*
London Stock Exchange (LSE)@*
Madrid Stock Exchange (MASE)@*
Manila Stock Exchange (MNSE)@*
Milan Stock Exchange/Bourse (MISE)@*
Montreal Stock Exchange (MOSE)@*
National Association of Securities Dealers Automated Quotation System (NASDAQ)@*
New York Coffee, Sugar and Cocoa Exchange (NYCSCE)@*
New York Commodity Exchange (COMEX)@*
New York Cotton Exchange (NYCE)@*
New York Mercantile Exchange (NYMEX)@*
New York Stock Exchange (NYSE)@*
Oslo Stock Exchange/Bourse (OSE)@*
Paris Stock Exchange/Bourse (PSE)@*
Securities Exchange of Thailand (SET)@*
Singapore International Monetary Exchange (SIMEX)@*
Singapore Stock Exchange (SSE)@*
Stockholm Stock Exchange/Bourse (STSE)@*
Tokyo Stock Exchange (TSE)@*
Toronto Stock Exchange (TOSE)@*
Winnipeg Commodity Exchange (WCE)@*
Zurich Stock Exchange/Bourse (ZSE)@*
@end[format]
@blankspace[3 lines]
________________________________________________________________
@heading[Country Codes (176)]
@begin[format]
AFGHANISTAN@*
ALBANIA @*
ALGERIA @*
AMERICAN-SAMOA@*
ANDORRA @*
ANGOLA  @*
ANGUILLA@*
ANTIGUA @*
ARGENTINA  @*
ARUBA@*
AUSTRALIA  @*
AUSTRIA @*
BAHAMAS @*
BAHRAIN @*
BANGLADESH @*
BARBADOS@*
BELGIUM @*
BELIZE  @*
BENIN@*
BERMUDA @*
BHUTAN  @*
BOLIVIA @*
BOTSWANA@*
BRAZIL  @*
BRITISH-VIRGIN-ISLANDS@*
BRUNEI  @*
BULGARIA@*
BURKINA-FASO@*
BURMA@*
BURUNDI @*
CAMEROON@*
CANADA  @*
CAPE-VERDE @*
CAYMAN-ISLANDS@*
CENTRAL-AFRICAN-REPUBLIC@*
CHAD @*
CHILE@*
CHINA@*
COLOMBIA@*
CONGO@*
COSTA-RICA @*
CUBA @*
CYPRUS  @*
CZECHOSLOVAKIA@*
DENMARK@*
DJIBOUTI@*
DOMINICA@*
DOMINICAN-REPUBLIC@*
EAST-GERMANY@*
ECUADOR @*
EGYPT@*
EL-SALVADOR@*
EQUATORIAL-GUINEA@*
ETHIOPIA@*
FIJI @*
FINLAND @*
FRANCE  @*
FRENCH-GUIANA@*
GABON@*
GAMBIA  @*
GHANA@*
GIBRALTAR  @*
GREECE  @*
GRENADA @*
GUADELOUPE @*
GUAM @*
GUATEMALA  @*
GUINEA-BISSAU@*
GUINEA  @*
GUYANA  @*
HAITI@*
HONDURAS@*
HONG-KONG  @*
HUNGARY @*
ICELAND @*
INDIA@*
INDONESIA  @*
IRAN @*
IRAQ @*
IRELAND @*
ISRAEL  @*
ITALY@*
IVORY-COAST@*
JAMAICA @*
JAPAN@*
JORDAN  @*
KAMPUCHEA  @*
KENYA@*
KUWAIT  @*
LAOS @*
LEBANON @*
LESOTHO @*
LIBERIA @*
LIBYA@*
LIECHTENSTEIN@*
LUXEMBOURG @*
MACAO@*
MADAGASCAR @*
MALAWI  @*
MALAYSIA@*
MALI @*
MALTA@*
MARTINIQUE @*
MAURITANIA @*
MAURITIUS  @*
MEXICO  @*
MONACO  @*
MOROCCO @*
MOZAMBIQUE @*
NAMIBIA @*
NEPAL@*
NETHERLANDS@*
NETHERLANDS-ANTILLES@*
NEW-CALEDONIA@*
NEW-ZEALAND@*
NICARAGUA  @*
NIGER@*
NIGERIA @*
NORTH-KOREA@*
NORWAY  @*
OMAN @*
PAKISTAN@*
PANAMA  @*
PAPUA-NEW-GUINEA@*
PARAGUAY@*
PERU @*
PHILIPPINES@*
POLAND  @*
PORTUGAL@*
QATAR@*
ROMANIA @*
RWANDA  @*
SAUDI-ARABIA@*
SENEGAL @*
SEYCHELLES @*
SIERRA-LEONE@*
SINGAPORE  @*
SOMALIA @*
SOUTH-AFRICA@*
SOUTH-KOREA@*
SPAIN@*
SRI-LANKA  @*
SUDAN@*
SURINAME@*
SWAZILAND  @*
SWEDEN  @*
SWITZERLAND@*
SYRIA@*
TAIWAN  @*
TANZANIA@*
THAILAND@*
TOGO @*
TONGA@*
TRINIDAD-TOBAGO@*
TUNISIA @*
TURKEY  @*
UAE  @*
UGANDA  @*
UK@*
URUGUAY @*
USA@*
USSR@*
US-VIRGIN-ISLANDS@*
VANUATU @*
VATICAN @*
VENEZUELA  @*
VIETNAM @*
WEST-GERMANY@*
WESTERN-SAMOA@*
YEMEN-ARAB-REPUBLIC@*
YEMEN-DEMO-REPUBLIC@*
YUGOSLAVIA @*
ZAIRE@*
ZAMBIA  @*
ZIMBABWE@*
@end[format]
@blankspace[3 lines]
________________________________________________________________
@heading[People Codes (269)]
@begin[itemize]
Argentina
@begin[itemize]
   President Raul Alfonsin  (ALFONSIN)

   Economy Minister Juan Sourrouille  (SOURROUILLE)

   Finance Secretary Mario Brodersohn  (BRODERSOHN)

   Central Bank Governor Jose Luis Machinea  (MACHINEA)
@end[itemize]

Australia
@begin[itemize]
   Prime Minister Robert (Bob) Hawke  (HAWKE)

   Federal Treasurer Paul Keating  (KEATING)

   Finance Minister Peter Walsh  (WALSH)

   Central Bank (Reserve Bank) Governor Bob Johnston  (JOHNSTON)
@end[itemize]

Austria
@begin[itemize]
   President Kurt Waldheim  (WALDHEIM)

   Chancellor Franz Vranitzky  (VRANITZKY)

   Finance Minister Ferdinand Lacina  (LACINA)

   Economy Minister Robert Graf  (GRAF)

   Central Bank President Stephan Koren  (KOREN)
@end[itemize]

Belgium
@begin[itemize]
   Prime Minister Wilfried Martens  (MARTENS)

   Deputy Prime Minister and Economy Minister Philippe Maystadt  (MAYSTADT)

   Finance Minister Mark Eyskens  (EYSKENS)

   Central Bank (National Bank) Governor Jean Godeaux  (GODEAUX)
@end[itemize]

Brazil
@begin[itemize]
   President Jose Sarney  (SARNEY)

   Finance Minister Mailson Nobrega (NOBREGA)

   (former) Finance Minister Luiz Carlos Bresser Pereira  (BRESSER-PEREIRA)

   Industry and Trade Minister Jose Hugo Castelo Branco  (CASTELO-BRANCO)

   Agriculture Minister Iris Rezende  (REZENDE)

   Mines and Energy Minister Aureliano Chaves  (CHAVES)

   Central Bank President Fernando Milliet  (MILLIET)
@end[itemize]

Canada
@begin[itemize]
   Prime Minister Brian Mulroney  (MULRONEY)

   Agriculture Minister John Wise  (WISE)

   Finance Minister Michael Wilson  (WILSON)

   Energy Minister Marcel Masse  (MASSE)

   (former) Central Bank (Bank of Canada) Governor Gerald K. Bouey  (BOUEY)

   Central Bank (Bank of Canada) Governor John Crow (CROW)
@end[itemize]

China
@begin[itemize]
   President Li Xiannian  (LI-XIANNIAN)

   Prime Minister Zhao Ziyang  (ZHAO-ZIYANG)

   Premier Li Peng (LI-PENG)

   Elder Statesman (a member of the Standing Committee of the
                   Communist Party Politburo) Deng Xiaoping  (DENG-XIAOPING)

   Finance Minister Wang Bingqian  (WANG-BINGQIAN)

   Foreign Economic Relations and Trade Minister Zheng Tuobin  (ZHENG-TUOBIN)

   Agriculture Minister He Kang  (HE-KANG)

   President of the People's Bank of China and Head of National
          Treasury Mrs Chen Muhua  (CHEN-MUHUA)
@end[itemize]

Cuba
@begin[itemize]
Fidel Castro, President of Cuba  (CASTRO)
@end[itemize]

Czechoslovakia
@begin[itemize]
   President Gustav Husak  (HUSAK)

   Prime Minister Lubomir Strougal  (STROUGAL)

   Deputy Minister and Minister of Agriculture and Food
        Miroslav Toman  (TOMAN)

   Finance Minister Jaromir Zak  (ZAK)
@end[itemize]

Denmark
@begin[itemize]
   Prime Minister Poul Schlueter  (SCHLUETER)

   Finance Minister Palle Simonsen  (SIMONSEN)

   (former) Economy Minister Anders Andersen  (ANDERSEN)

   Economy Minister Knud Enggaard (ENGGAARD)

   Energy Minister Svend Erik Hovmand  (HOVMAND)

   (former) Agriculture Minister Mrs Britta Schall Holberg  (HOLBERG)
   
   Agriculture Minister Laurits Toernaes (TOERNAES)

   Central Bank Governor Erik Hoffmeyer  (HOFFMEYER)
@end[itemize]

East Germany
@begin[itemize]
Head of State and Chairman of Council of State Erich Honecker, who is also
General Secretary of the Socialist Unity Party  (HONECKER)

   Prime Minister Willi Stoph  (STOPH)

   Finance Minister Ernst Hoefner  (HOEFNER)

   Agriculture, Forestry and Food Minister Bruno Leitz  (LEITZ)

   State Bank President Horst Kaminsky  (KAMINSKY)
@end[itemize]

Egypt
@begin[itemize]
   President Hosni Mubarak  (MUBARAK)

   Prime Minister Atef Sedki  (SEDKI)

   (former) Economy Minister Youssri Wali  (WALI)

   Economy Minister Youssri mustapha  (MUSTAPHA)

   Finance Minister Mohammed Ahmed al-Razaz  (MOHAMMED-AHMED-AL-RAZAZ)

   Oil and Mineral Wealth Minister Abdel-Hadi Kandeel (ABDEL-HADI-KANDEEL)

   Central Bank Governor Mohammed Salaheddin Hamid  (MOHAMMED-SALAHEDDIN-HAMID)
@end[itemize]

Finland
@begin[itemize]
   President Mauno Koivisto  (KOIVISTO)

   Prime Minister Harri Holkeri  (HOLKERI)

   Finance Minister Erkki Liikanen  (LIIKANEN)

   Trade and Industry Minister Ilkka Suominen  (SUOMINEN)

   Central Bank Governor Rolf Kullberg  (KULLBERG)
@end[itemize]


France
@begin[itemize]
   President Francois Mitterrand  (MITTERRAND)

   Prime Minister Jacques Chirac  (CHIRAC)

   Minister of State for Economy, Finance and Privatisation  
       Edouard Balladur  (BALLADUR)

   Agriculture Minister Francois Guillaume  (GUILLAUME)

   Central Bank (Bank of France) Governor Jacques de Larosiere  
                                            (DE-LAROSIERE)
@end[itemize]

Greece
@begin[itemize]
   President Christos Sartzetakis  (SARTZETAKIS)

   Prime Minister Andreas Papandreou  (PAPANDREOU)

   Economy Minister Panayotis Roumeliotis (ROUMELIOTIS)

   (former) Economy Minister Kostas Simitis  (SIMITIS)

   Finance Minister Dimitris Tsovolas  (TSOVOLAS)

   Agriculture Minister Yannis Pottakis  (POTTAKIS)

   Central Bank Governor Dimitris Halikias  (HALIKIAS)
@end[itemize]

Hungary
@begin[itemize]
   Head of State and President of Presidential Council Karoly Nemeth  (NEMETH)

   Prime Minister Karoly Grosz  (GROSZ)

   Finance Minister Miklos Villanyi (VILLANYI)

   (former) Finance Minister Peter Medgyessy  (MEDGYESSY)

   Agriculture and Food Minister Jenoe Vancsa  (VANCSA)

   Central Bank President Dr Matyas Timar  (TIMAR)
@end[itemize]

Iceland
@begin[itemize]
   President Mrs Vigdis Finnbogadottir  (FINNBOGADOTTIR)

   Prime Minister Thorsteinn Palsson  (PALSSON)

   Finance Minister Jon Baldvin Hannibalsson  (HANNIBALSSON)

   Central Bank Chairman Jonas G. Rafnar  (RAFNAR)
@end[itemize]

India
@begin[itemize]
   President Ramaswamy Venkataraman  (VENKATARAMAN)

   Prime Minister Rajiv Gandhi  (GANDHI)

   Finance and Commerce Minister Narain Dutt Tiwari  (TIWARI)

   Energy Minister Vasant Sathe  (SATHE)

   Agriculture and Rural Development Minister Gurdial Singh Dhillon  
                                                     (DHILLON)

   Central Bank (Reserve Bank) Governor R.N. Malhotra  (MALHOTRA)
@end[itemize]

Indonesia
@begin[itemize]
   President General Suharto  (SUHARTO)

   Economy Minister Ali Wardhana  (WARDHANA)

   Finance Minister Radius Prawiro  (PRAWIRO)

   Mining and Energy Minister Subroto  (SUBROTO)

   Central Bank Governor Arifin Siregar  (SIREGAR)
@end[itemize]

Iran
@begin[itemize]
   Revolutionary Leader Ayatollah Ruhollah Khomeini  (KHOMEINI)

   President Hojatoleslam Ali Khameini (KHAMEINI)

   Prime Minister Mir-Hossein Mousavi  (MOUSAVI)

   Oil Minister Gholamreza Aqazadeh  (AQAZADEH)

   Central Bank Governor Majid Qassemi  (QASSEMI)

   Speaker of the Majlis (Parliament) Hojatoleslam Ali Hashemi Rafsanjani  
                                        (RAFSANJANI)
@end[itemize]

Ireland
@begin[itemize]
   President Patrick Hillery  (HILLERY)

   Prime Minister Charles Haughey  (HAUGHEY)

   Finance Minister Ray MacSharry  (MACSHARRY)

   Agriculture Minister Michael O'Kennedy  (O-KENNEDY)

   Central Bank Governor Tomas O'Cofaigh  (O-COFAIGH)
@end[itemize]

Italy
@begin[itemize]
   President Francesco Cossiga  (COSSIGA)

   Prime Minister Giovanni Goria  (GORIA)

   Deputy Prime Minister and Treasury Minister Guiliano Amato  (AMATO)

   Finance Minister Antonio Gava  (GAVA)

   Budget and Planning Minister Emilio Colombo  (COLOMBO)

   Agriculture and Forestry Minister Filippo Maria Pandolfi  (PANDOLFI)

   Central Bank (Bank of Italy) Governor Carlo Ciampi  (CIAMPI)
@end[itemize]



Japan
@begin[itemize]
   Prime Minister Noboru Takeshita (TAKESHITA)

   (former) Prime Minister Yasuhiro Nakasone  (NAKASONE)

   Finance Minister Kiichi Miyazawa  (MIYAZAWA)

   International Trade and Industry Minister Hajime Tamura  (TAMURA)

   Director-General of Economic Planning Agency Eiichi Nakao (NAKAO)

   (former) Director-General of Economic Planning Agency Tetsuo Kondo  (KONDO)

   Central Bank (Bank of Japan) Governor Satoshi Sumita  (SUMITA)
@end[itemize]

Libya
@begin[itemize]
Muammar Gaddafi, Libyan leader  (GADDAFI)
@end[itemize]


Luxembourg
@begin[itemize]
   Prime Minister (who is also Minister of Communications, of Finance,
                  of Planning and of Posts, Telephone and Telegraph)
                  Jacques Santer  (SANTER)
@end[itemize]

Mexico
@begin[itemize]
   President Miguel de la Madrid  (DE-LA-MADRID)

   Finance Minister Gustavo Petricioli  (PETRICIOLI)

   Oil, Mines and Parastatal Industry Minister Alfredo Del Mazo  (DEL-MAZO)

   (former) Director-General of Pemex (State Oil Company) Ramon Beteta  (BETETA)

   Director-General of Pemex (State Oil Company) francisco Rojas  (ROJAS)

   Central Bank President Miguel Mancera Aguayo (MANCERA-AGUAYO)
@end[itemize]


The Netherlands
@begin[itemize]
   Prime Minister Ruud Lubbers  (LUBBERS)

   Deputy Prime Minister and Minister for Economic Affairs
        Rudolf De Korte  (DE-KORTE)

   Finance Minister Onno Ruding  (RUDING)

   Agriculture Minister Gerrit Braks  (BRAKS)

   Central Bank Governor Wim Duisenberg  (DUISENBERG)
@end[itemize]


New Zealand
@begin[itemize]
   Prime Minister David Lange  (LANGE)

   Finance Minister Roger Douglas  (DOUGLAS)

   Agriculture Minister Colin Moyle  (MOYLE)

   Central Bank (Reserve Bank) Governor Spencer Russell  (RUSSELL)
@end[itemize]

Nicaragua
@begin[itemize]
Daniel Ortega, President of Nicaragua  (ORTEGA)
@end[itemize]

Nigeria
@begin[itemize]
   President General Ibrahim Babangida  (BABANGIDA)

   Agriculture Minister Major-General Mohammed Gado Nasko  (NASKO)

   Finance Minister Chu Okongwu  (OKONGWU)

   Petroleum Resources (Oil) Minister Rilwanu Lukman  (LUKMAN)

   Central Bank Governor Alhaji Abdul Ahmed  (ALHAJI-ABDUL-AHMED)
@end[itemize]

Norway
@begin[itemize]
   Prime Minister Mrs Gro Harlem Brundtland  (BRUNDTLAND)

   Finance Minister Gunnar Berge  (BERGE)

   Oil and Energy Minister Arne Oeien  (OEIEN)

   Central Bank Governor Hermud Skaanland  (SKAANLAND)
@end[itemize]


Pakistan
@begin[itemize]
   President General Zia-ul-Haq  (ZIA-UL-HAQ)

   Prime Minister Mohammad Khan Junejo  (MOHAMMAD-KHAN-JUNEJO)

   Finance and Economic Affairs, Petroleum and Natural Resources
         Minister  Mohammad Yasin Khan Wattoo  
                                        (MOHAMMAD-YASIN-KHAN-WATTOO)

   Agriculture Minister Mohammad Ibrahim Jaffrey Baluch  
                                      (MOHAMMAD-IBRAHIM-JAFFREY-BALUCH)

   Central Bank (State Bank) Governor Wasim Aun Jaffrey  (WASIM-AUN-JAFFREY)
@end[itemize]

Peru
@begin[itemize]
   President Alan Garcia  (GARCIA)

   Economy and Finance Minister Gustavo Saberbein Chevalier  (SABERBEIN)

   (former) Trade and Industry Minister Manuel Romero Caro  (CARO)

   Industry and Trade Minister Alberto Vera La Rose (VERA-LA-ROSA)

   Energy Minister Abel Salinas (SALINAS)

   Agriculture Minister Remigio Morales Bermudez  (MORALES-BERMUDEZ)

   Central Bank President Cesar Ferrari  (FERRARI)
@end[itemize]

Philippines
@begin[itemize]
   President Mrs Corazon Aquino  (AQUINO)

   (former) Finance Minister Jaime Ongpin  (ONGPIN)

   Finance Minister Vicente Jayme (JAYME)

   Agriculture Minister Carlos Dominguez  (DOMINGUEZ)

   Trade and Industry Minister Jose Concepcion  (CONCEPCION)

   Central Bank Governor Jose Fernandez  (FERNANDEZ)
@end[itemize]

Poland
@begin[itemize]
Head of State and Chairman of Council of State (who is also First Secretary
of Polish United Workers (Communist) Party) General Wojciech Jaruzelski
(JARUZELSKI)

   Prime Minister Zbigniew Messner  (MESSNER)

   Finance Minister Bazyli Samojlik  (SAMOJLIK)
@end[itemize]




Portugal
@begin[itemize]
   President Mario Soares  (SOARES)

   Prime Minister Anibal Cavaco Silva  (CAVACO-SILVA)

   Finance Minister Miguel Riberio Cadilhe  (RIBERIO-CADILHE)

   Agriculture, Fisheries and Food Minister Alvaro Barreto  (BARRETO)

   Central Bank Governor Jose Tavares Moreia  (TAVARES-MOREIA)
@end[itemize]


Saudi Arabia
@begin[itemize]
   Head of State and also Prime Minister King Fahd (Ibn Abdulaziz)  
                                                         (KING-FAHD)

   Finance and National Economy Minister Sheikh Mohammed Ali Abal-Khail
                                               (MOHAMMED-ALI-ABAL-KHAIL)

   Petroleum/Oil Minister Hisham Nazer  (HISHAM-NAZER)

   Governor of the Saudi Arabian Monetary Agency Hamad Saud al-Sayyari
                                                 (HAMAD-SAUD-AL-SAYYARI)
@end[itemize]

South Africa
@begin[itemize]
   President P.W. Botha  (BOTHA)

   Finance Minister Barend du Plessis  (DU-PLESSIS)

   Central Bank (Reserve Bank) Governor Gerhard de Kock  (DE-KOCK)
@end[itemize]

Spain
@begin[itemize]
   Prime Minister Felipe Gonzalez  (GONZALEZ)

   Economy, Trade and Finance Minister Carlos Solchaga  (SOLCHAGA)

   Agriculture Minister Carlos Romero  (ROMERO)

   Central Bank Governor Mariano Rubio  (RUBIO)
@end[itemize]


Sweden
@begin[itemize]
   Prime Minister Ingvar Carlsson  (CARLSSON)

   Finance Minister Kjell-Olof Feldt  (FELDT)

   Central Bank Governor Bengt Dennis  (DENNIS)
@end[itemize]

Switzerland
@begin[itemize]
   Economy Minister Jean-Pascal Delamuraz  (DELAMURAZ)

   Finance Minister Otto Stich  (STICH)

   Central Bank President Pierre Languetin (LANGUETIN)
@end[itemize]

Taiwan
@begin[itemize]
   President Lee Teng-hui (LEE-TENG-HUI)

   (former) President Chiang Ching-kuo  (CHIANG-CHING-KUO)

   Prime Minister Yu Kuo-hua  (YU-KUO-HUA)

   Economy Minister Lee Ta-hai  (LEE-TA-HAI)

   Finance Minister Robert Chien  (CHIEN)
@end[itemize]

Thailand
@begin[itemize]
   President General Prem Tinsulanonda  (TINSULANONDA)

   Finance Minister Suthee Singhasaneh  (SINGHASANEH)

   Agriculture Minister General Harn Leenanon  (LEENANON)
@end[itemize]

Turkey
@begin[itemize]
   President Kenan Evren  (EVREN)

   Prime Minister Turgut Ozal  (OZAL)

   Finance Minister Ahmet Kurtcebe Alptemocin  (ALPTEMOCIN)

   Central Bank Governor Rusdu Saracoglu  (SARACOGLU)
@end[itemize]


United Kingdom
@begin[itemize]
   Prime Minister Margaret Thatcher  (THATCHER)

   Chancellor of the Exchequer (Finance Minister) Nigel Lawson  (LAWSON)

   Energy Secretary Cecil Parkinson  (PARKINSON)

   Trade and Industry Secretary Lord Young  (YOUNG)

   Central Bank (Bank of England) Governor Robin Leigh-Pemberton  
                                                  (LEIGH-PEMBERTON)
@end[itemize]


United States
@begin[itemize]
   President Ronald Reagan  (REAGAN)

   Treasury Secretary James Baker  (JAMES-BAKER)

   Commerce Secretary C. William Verity  (VERITY)

   Energy Secretary John Herrington  (HERRINGTON)

   Agriculture Secretary Richard Lyng  (LYNG)

   White House Chief of Staff Howard Baker  (HOWARD-BAKER)

   Representative for Trade Negotiations Clayton Yeutter  (YEUTTER)

   Director of Management and Budget Office James Miller  (JAMES-MILLER)

   Chairman of Federal Reserve Board Alan Greenspan  (GREENSPAN)

   President of New York Federal Reserve Bank E. Gerald Corrigan  
                                                    (CORRIGAN)
@end[itemize]

USSR (Soviet Union)
@begin[itemize]
   President Andrei Gromyko  (GROMYKO)

   General Secretary of the Communist Party Mikhail Gorbachev  (GORBACHEV)

   Prime Minister Nikolai Ryzhkov  (RYZHKOV)

   Finance Minister Boris Gostyev  (GOSTYEV)

   State Bank Board Chairman Viktor Dementsev  (DEMENTSEV)
@end[itemize]

West Germany
@begin[itemize]
   President Richard Von Weizsaecker  (VON-WEIZSAECKER)

   Chancellor Helmut Kohl  (KOHL)

   Finance Minister Gerhard Stoltenberg  (STOLTENBERG)

   Economy Minister Martin Bangemann  (BANGEMANN)

   Agriculture, Food and Forestry Minister Ignaz Kiechle  (KIECHLE)

   Central Bank (Bundesbank) President Karl Otto Poehl  (POEHL)
@end[itemize]

Yugoslavia
@begin[itemize]
   Prime Minister Branko Mikulic  (MIKULIC)

   Finance Minister Svetozar Rikanovic  (RIKANOVIC)

   Central Bank (National Bank) Governor Dusan Vlatkovic  (VLATKOVIC)
@end[itemize]

European Community Commission
@begin[itemize]
   President Jacques Delors  (DELORS)

   Commissioner for Agriculture and Forestry Frans Andriessen  (ANDRIESSEN)

   Commissioner for Trade Policy Willy de Clercq  (DE-CLERCQ)
@end[itemize]

International Monetary Fund Managing Director Michel Camdessus (CAMDESSUS)

World Bank President Barber Conable  (CONABLE)

International Atomic Energy Agency Director-General Hans Blix  (BLIX)

International Energy Agency Executive Director Helga Steeg  (STEEG)

Asian Development Bank President Masao Fujioka  (FUJIOKA)

United Nations Secretary-General Javier Perez de Cuellar  (PEREZ-DE-CUELLAR)

UNCTAD (United Nations Conference on Trade and Development)
    Secretary-General Kenneth Dadzie  (DADZIE)

Organisation for Economic Cooperation and Development
    Secretary-General Jean-Claude Paye  (PAYE)

International Air Transport Association Director General
    Guenter Eser  (ESER)

Securities and Exchange Commission Chairman David S. Ruder  (RUDER)

General Agreement on Tariffs and Trade Director-General 
     Arthur Dunkel  (DUNKEL)

Milton Friedman, economist   (FRIEDMAN)

Henry Kaufman, economist with Salomon Bros.  (KAUFMAN)

Ivan Boesky  (BOESKY)

Carl Icahn  (ICAHN)

T. Boone Pickens  (PICKENS)

Rupert Murdoch, publisher and chairman of News Corp Ltd  (MURDOCH)

Robert Maxwell, publisher and chairman of Mirror Group of newspapers (MAXWELL)

Roland (Tiny) Rowland, chairman of Lonrho Plc  (ROWLAND)

Asher Edelman  (EDELMAN)

Marc Rich  (RICH)

Alan Bond, chairman of Bond Corp Ltd  (BOND)

Jorio Dauster, President of The Brazilian Coffee Institute  (DAUSTER)

Beryl Sprinkel, Chairman of Council of Economic Advisers  (SPRINKEL)

Yasser Arafat, Chairman of Palestine Liberation Organisation  (ARAFAT)

Paul Volcker, economist  (VOLCKER)

Preston Martin, member of Federal Reserve Board  (MARTIN)
@end[itemize]
 





